In [ ]:
import numpy as np
import pandas as pd
from ast import literal_eval
from keras.models import model_from_json

In [ ]:
def is_less_than(v1,v2):
    return np.sum(v1) < np.sum(v2)

In [ ]:
def is_equal(v1,v2):
    return np.isclose(np.sum(v1),np.sum(v2))

In [ ]:
def check_rule(v1,v2):
    return is_less_than(v1,v2) or is_equal(v1,v2)

In [ ]:
def is_a_valid_pair(v1,v2):
    for idx in range(1,len(v1)): #ignoring last sum because all vectors sum 1
        if not check_rule(v1[:idx], v2[:idx]):
            return False
    return True

In [ ]:
def convert_string_to_list(column):
    # return column.apply(lambda cell: np.fromstring(cell[1:-1], dtype=np.float64, sep=' '))
    return column.apply(lambda cell: np.array(literal_eval(cell)))

In [ ]:
def train_test_split(df, train_percent=.8):
    n_samples = df.shape[0]
    perm = np.random.permutation(n_samples)
    train_end = int(train_percent * n_samples)
    train = df.iloc[perm[:train_end]]
    test = df.iloc[perm[train_end:]]
    return train, test

In [ ]:
def read_csv(filename):
    df_samples = pd.read_csv(filename,sep=';',names=['V1','V2','V1V2','V2V1'])
    df_samples['V1'] = convert_string_to_list(df_samples['V1'])
    df_samples['V2'] = convert_string_to_list(df_samples['V2'])
    # df_samples.head()
    return df_samples

In [ ]:
def format_input(df_samples):
    df_input = df_samples.filter(['V1','V2','V1V2']).rename(columns={'V1': 'A', 'V2': 'B', 'V1V2': 'AB'})
    df_input = pd.concat([df_input,df_samples.filter(['V2','V1','V2V1']).rename(columns={'V2':'A', 'V1': 'B', 'V2V1': 'AB'})])
    return df_input

In [ ]:
def read_file_format_and_split(filename):
    df_samples = read_csv(filename)
    df_input = format_input(df_samples)
    return train_test_split(df_input)

In [ ]:
def load_model(dim, optimizer):
    model_filename = './results/model/seq_200relu_100relu_50ep__d'+str(dim)+'_opt-'+optimizer
    json_file = open(model_filename+'.json')
    loaded_model_json = json_file.read()
    json_file.close()

    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights(model_filename+'.h5')
    loaded_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return loaded_model

In [ ]:
def save_model_json(model,config_name,dim,opt):
    model_json = model.to_json()
    model_name = 'results/model/'+config_name+'_d'+str(dim)+'_opt-'+opt+'.json'
    with open(model_name,'w') as json_file:
        json_file.write(model_json)

In [ ]:
def save_model_weights(model,config_name,dim,opt):
    model_weights_name = 'results/model/'+config_name+'_d'+str(dim)+'_opt-'+opt+'.h5'
    model.save_weights(model_weights_name)

In [ ]:
def save_data_to_plot(output_name,optimizer,dim,epoch,value):
    with open(output_name,'a') as plot_file:
        plot_file.write(optimizer+';'+str(dim)+';'+str(epoch)+';'+str(value)+'\n')

In [ ]:
def save_data(output_name,optimizer,dim,value):
    with open(output_name,'a') as plot_file:
        plot_file.write(optimizer+';'+str(dim)+';'+str(value)+'\n')